In [1]:
%load_ext autoreload
%autoreload 2

%cd /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/funcmol/notebooks

/datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/funcmol/notebooks


In [2]:
import os
import sys
import torch
from pathlib import Path
from lightning import Fabric

# 设置 torch.compile 兼容性
try:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
except ImportError:
    # PyTorch 版本 < 2.0 不支持 torch._dynamo
    print("Warning: torch._dynamo not available in this PyTorch version")

## set up environment
# 当前目录是 funcmol/notebooks，需要将 funcmol 的父目录添加到路径中
# 这样 funcmol 才能作为包被正确导入
notebook_dir = Path(os.getcwd())  # funcmol/notebooks
project_root = notebook_dir.parent.parent  # funcmol 的父目录
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

from funcmol.dataset.dataset_field import create_gnf_converter, prepare_data_with_sample_idx
from funcmol.utils.utils_nf import load_neural_field
from funcmol.utils.utils_fm import load_checkpoint_fm
from funcmol.utils.constants import PADDING_INDEX
from funcmol.utils.gnf_visualizer import (
    visualize_1d_gradient_field_comparison, 
    GNFVisualizer,
    visualize_generation_step,
    visualize_generated_molecule
)
from funcmol.utils.misc import load_nf_config, load_funcmol_config, create_field_function

# 模型根目录
model_root = "/datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/neural_field"

Project root: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield
Python path: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield


In [3]:
# TODO：手动指定是 gt_only、gt_pred 还是 denoiser_only 模式
option = 'denoiser_only'  # 'gt_only', 'gt_pred', 'denoiser_only'

# TODO：手动指定 checkpoint 文件路径，会根据ckpt_path自动提取exp_name
nf_ckpt_path = '/datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_qm9/20251024/lightning_logs/version_0/checkpoints/model-epoch=409.ckpt'
fm_ckpt_path = '/datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/lightning_logs/version_0/checkpoints/model-epoch=399.ckpt'
# nf_ckpt_path = '/datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_qm9/20250911/lightning_logs/version_1/checkpoints/model-epoch=39.ckpt'
# fm_ckpt_path = '/datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20250917/lightning_logs/version_22/checkpoints/model-epoch=144.ckpt'

# TODO：手动指定 sample_idx（仅用于 gt_only 和 gt_pred 模式）
sample_idx = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if option == 'denoiser_only':
    # 对于 denoiser_only 模式，使用 FuncMol 的路径
    ckpt_parts = Path(fm_ckpt_path).parts
    funcmol_idx = ckpt_parts.index('funcmol')
    exp_name = f"{ckpt_parts[funcmol_idx + 1]}/{ckpt_parts[funcmol_idx + 2]}"  # fm_qm9/20250912
    ckpt_name = Path(fm_ckpt_path).stem  # funcmol-epoch=319
    model_dir = os.path.join("/datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol", exp_name)
    output_dir = os.path.join(model_dir, ckpt_name)
    os.makedirs(output_dir, exist_ok=True)
    print(f"Option: {option}")
    print(f"FuncMol model directory: {model_dir}")
    print(f"FuncMol checkpoint: {ckpt_name}")
    print(f"Neural Field checkpoint: {nf_ckpt_path}")
    print(f"Output directory: {output_dir}")
else:
    # 对于 gt_only 和 gt_pred 模式，使用 Neural Field 的路径
    ckpt_parts = Path(nf_ckpt_path).parts
    neural_field_idx = ckpt_parts.index('neural_field')
    exp_name = f"{ckpt_parts[neural_field_idx + 1]}/{ckpt_parts[neural_field_idx + 2]}"  # nf_qm9/20250911
    ckpt_name = Path(nf_ckpt_path).stem  # model-epoch=39
    model_dir = os.path.join(model_root, exp_name)
    output_dir = os.path.join(model_dir, ckpt_name)
    os.makedirs(output_dir, exist_ok=True)
    print(f"Option: {option}")
    print(f"Model directory: {model_dir}")
    print(f"Checkpoint: {ckpt_name}")
    print(f"Output directory: {output_dir}")

Option: denoiser_only
FuncMol model directory: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111
FuncMol checkpoint: model-epoch=399
Neural Field checkpoint: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_qm9/20251024/lightning_logs/version_0/checkpoints/model-epoch=409.ckpt
Output directory: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399


In [4]:
## Load data
fabric = Fabric(
    accelerator="auto",
    devices=1,
    precision="32-true",
    strategy="auto"
)
fabric.launch()

# 使用 load_nf_config 函数从 configs 目录加载配置
config = load_nf_config("train_nf_qm9")

if option == 'denoiser_only':
    # 对于 denoiser_only 模式，不需要加载数据集
    batch, gt_coords, gt_types = None, None, None
    print("Denoiser-only mode: No dataset loading required")
else:
    # 准备包含特定样本的数据
    batch, gt_coords, gt_types = prepare_data_with_sample_idx(config, device, sample_idx)
    print(f"Data loaded for sample {sample_idx}: {gt_coords.shape}, {gt_types.shape}")

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Dataset directory: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/funcmol/dataset/data
Config loaded successfully: train_nf_qm9
n_iter from converter config: 2000
Denoiser-only mode: No dataset loading required


In [5]:
# 修复后的模型加载代码 - 使用YAML配置文件加载参数
print(f"\nProcessing model from: {model_dir}")

## Load model
if option == 'denoiser_only':
    # 加载 Neural Field 模型和 FuncMol 模型
    from funcmol.models.funcmol import FuncMol
    
    # 加载 Neural Field 模型
    print(f"Loading Neural Field model from: {nf_ckpt_path}")
    encoder, decoder = load_neural_field(nf_ckpt_path, config)
    # 确保模型在正确的设备上
    encoder = encoder.cuda()
    decoder = decoder.cuda()
    encoder.eval()
    decoder.eval()
    
    # 使用YAML配置文件加载FuncMol配置
    funcmol_config = load_funcmol_config("train_fm_qm9", config)
    
    # 创建FuncMol模型
    funcmol = FuncMol(funcmol_config)
    funcmol = funcmol.cuda()
    
    # 加载checkpoint
    funcmol, _ = load_checkpoint_fm(funcmol, fm_ckpt_path, fabric=fabric)
    funcmol.eval()
    
    print(">> FuncMol model loaded successfully!")
    
    # 使用统一的场计算函数
    field_func = create_field_function(
        mode='denoiser',
        decoder=decoder,
        funcmol=funcmol,
        config=config
    )
    codes = None  # denoiser 模式不需要预计算的 codes
    
elif option == 'gt_pred':
    # 使用手动指定的 checkpoint 文件路径
    if not os.path.exists(nf_ckpt_path):
        raise FileNotFoundError(f"Checkpoint file not found: {nf_ckpt_path}")
    
    print(f"Loading model from: {nf_ckpt_path}")
    encoder, decoder = load_neural_field(nf_ckpt_path, config)
    
    # 确保模型在正确的设备上
    encoder = encoder.cuda()
    decoder = decoder.cuda()
    encoder.eval()
    decoder.eval()
    
    # 生成 codes
    print(f"Batch device: {batch.pos.device}")
    print(f"Encoder device: {next(encoder.parameters()).device}")
    with torch.no_grad():
        codes = encoder(batch)
    # 使用统一的场计算函数
    field_func = create_field_function(
        mode='predicted',
        decoder=decoder,
        codes=codes
    )
else:  # gt only
    encoder, decoder = None, None
    codes = None

converter = create_gnf_converter(config)

# 创建场函数（在converter定义之后）
if option == 'gt_only':
    field_func = create_field_function(
        mode='gt',
        converter=converter,
        gt_coords=gt_coords,
        gt_types=gt_types
    )
print(f"Model loaded successfully!")


Processing model from: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111
Loading Neural Field model from: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_qm9/20251024/lightning_logs/version_0/checkpoints/model-epoch=409.ckpt
Loading Lightning checkpoint from: /datapool/data2/home/pxg/data/hyc/funcmol-main-neuralfield/exps/neural_field/nf_qm9/20251024/lightning_logs/version_0/checkpoints/model-epoch=409.ckpt
>> loaded dec
>> loaded enc
Model loaded successfully!
Loading configuration from YAML: train_fm_qm9
>> Using diffusion_method: new_x0
>> DDPM config: {'hidden_dim': 128, 'num_layers': 4, 'time_emb_dim': 64, 'dropout': 0.1, 'num_timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02, 'schedule': 'linear', 's1': 0.008, 'sT': 0.008, 'w': 1.0}
>> loaded denoiser
>> loaded model trained for 399 epochs
>> FuncMol model loaded successfully!
Model loaded successfully!


In [ ]:
if option == 'denoiser_only':
    from funcmol.models.funcmol import FuncMol
    from funcmol.utils.utils_fm import load_checkpoint_fm
    from omegaconf import OmegaConf

    # ========== 配置codes加载方式 ==========
    codes_source = 'load'  # 'load' 或 'sample'，'load' 表示从保存的文件加载，'sample' 表示随机采样
    
    # 如果选择 'load'，可以通过以下两种方式指定：
    # 方式1：指定codes文件索引（0, 1, 2, 3, 4等），会自动构建路径
    codes_idx = 0  # 例如：0 表示 code_0000_tanh.pt, 1 表示 code_0001_tanh.pt
    
    # 方式2：直接指定codes文件的完整路径（如果指定了，会优先使用此路径）
    codes_file_path = None  # 例如：'/path/to/code_0000_tanh.pt' 或 None
    
    # field_method 需要与 sample_fm.py 中保存codes时使用的field_method一致
    field_method = 'tanh'  # 与 sample_fm.py 中的 field_methods = ['tanh'] 一致
    # ======================================

    # 确保与当前NF配置一致（保持decoder/encoder/dset对齐）
    funcmol_config["encoder"] = OmegaConf.to_container(config.encoder, resolve=True)
    funcmol_config["decoder"] = OmegaConf.to_container(config.decoder, resolve=True)
    funcmol_config["dset"] = OmegaConf.to_container(config.dset, resolve=True)

    funcmol_ddpm = FuncMol(funcmol_config)
    funcmol_ddpm = funcmol_ddpm.cuda()
    funcmol_ddpm, code_stats = load_checkpoint_fm(funcmol_ddpm, fm_ckpt_path)
    funcmol_ddpm.eval()

    # 同步 code_stats 到 decoder，保证数值尺度一致
    try:
        decoder.set_code_stats(code_stats)
    except Exception:
        pass

    # 使用与 sample_fm.py 完全一致的配置加载方式
    # 关键：使用 Hydra 加载 sample_fm.yaml 配置，确保和 sample_fm.py 完全一致
    import hydra
    from omegaconf import DictConfig
    
    # 使用 Hydra 加载 sample_fm.yaml 配置（与 sample_fm.py 完全相同的方式）
    configs_dir = project_root / "funcmol" / "configs"
    with hydra.initialize_config_dir(config_dir=str(configs_dir), version_base=None):
        sample_fm_config = hydra.compose(config_name="sample_fm")
    
    # 转换为字典格式（与 sample_fm.py 第51行完全一致）
    config_dict = OmegaConf.to_container(sample_fm_config, resolve=True)
    
    # 构建与 sample_fm.py 完全相同的配置结构（第149-150行）
    method_config = config_dict.copy()
    method_config['converter']['gradient_field_method'] = field_method
    
    # 创建 converter（与 sample_fm.py 第151行完全一致）
    converter = create_gnf_converter(method_config)
    
    # 打印配置信息以便调试和验证
    converter_config = method_config['converter']
    print(f"\n=== Converter 配置信息（用于验证） ===")
    print(f"n_iter: {converter_config.get('n_iter')}")
    print(f"全局 eps: {converter_config.get('eps')}, min_samples: {converter_config.get('min_samples')}")
    if 'method_configs' in converter_config and field_method in converter_config['method_configs']:
        method_specific = converter_config['method_configs'][field_method]
        print(f"方法特定配置 ({field_method}):")
        print(f"  eps: {method_specific.get('eps', 'N/A')}")
        print(f"  min_samples: {method_specific.get('min_samples', 'N/A')}")
        print(f"  n_query_points: {method_specific.get('n_query_points', 'N/A')}")
        print(f"  step_size: {method_specific.get('step_size', 'N/A')}")
    print("=" * 50)

    # 获取 codes 的维度信息
    grid_size = method_config.get('dset', {}).get('grid_size', 9)  # 与 sample_fm.py 一致
    code_dim = method_config.get('encoder', {}).get('code_dim', 128)  # 与 sample_fm.py 一致
    
    # 根据配置加载或采样 codes
    print(f"\n=== Codes 加载配置信息 ===")
    print(f"codes_source: {codes_source}")
    print(f"codes_file_path: {codes_file_path}")
    print(f"codes_idx: {codes_idx}")
    print(f"field_method: {field_method}")
    print(f"model_dir: {model_dir}")
    
    if codes_source == 'load':
        # 从保存的codes文件加载
        if codes_file_path is not None:
            # 使用直接指定的路径
            code_path = Path(codes_file_path)
            print(f"使用直接指定的codes文件路径")
        else:
            # 根据索引自动构建路径
            # sample_fm.py 保存codes的路径格式：{model_dir}/molecule/code_{generated_idx:04d}_{field_method}.pt
            mol_save_dir = Path(model_dir) / "molecule"
            code_path = mol_save_dir / f"code_{codes_idx:04d}_{field_method}.pt"
            print(f"根据索引自动构建路径:")
            print(f"  mol_save_dir: {mol_save_dir}")
            print(f"  文件名格式: code_{codes_idx:04d}_{field_method}.pt")
        
        print(f"\n最终使用的codes路径: {code_path}")
        print(f"路径是否存在: {code_path.exists()}")
        
        if not code_path.exists():
            raise FileNotFoundError(
                f"Codes file not found: {code_path}\n"
                f"Please check if the file exists or run sample_fm.py first to generate codes."
            )
        
        print(f"Loading codes from: {code_path}")
        codes = torch.load(code_path, map_location='cuda')
        # 确保codes的形状正确 [1, grid_size^3, code_dim]
        if codes.dim() == 2:
            # 如果是 [grid_size^3, code_dim]，添加batch维度
            codes = codes.unsqueeze(0)
        print(f"Loaded codes shape: {codes.shape}")
        
    else:
        # 随机采样 codes
        print("Sampling codes using DDPM...")
        with torch.no_grad():
            codes = funcmol_ddpm.sample_ddpm(shape=(1, grid_size**3, code_dim), progress=False)
        print(f"Sampled codes shape: {codes.shape}")

    # 使用统一的场计算函数
    field_func = create_field_function(
        mode='ddpm',
        decoder=decoder,
        codes=codes
    )
    print("Codes loaded/sampled and field_func set.")

>> loaded denoiser
>> loaded model trained for 399 epochs

=== Converter 配置信息（用于验证） ===
n_iter: 2000
全局 eps: None, min_samples: None
方法特定配置 (tanh):
  eps: 0.15
  min_samples: 30
  n_query_points: 1000
  step_size: 0.1

=== Codes 加载配置信息 ===
codes_source: load
codes_file_path: None
codes_idx: 0
field_method: tanh
model_dir: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111
根据索引自动构建路径:
  mol_save_dir: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/molecule
  文件名格式: code_0000_tanh.pt

最终使用的codes路径: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/molecule/code_0000_tanh.pt
路径是否存在: True
Loading codes from: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/molecule/code_0000_tanh.pt
Loaded codes shape: torch.Size([1, 729, 128])
Codes loaded/sampled and field_func set.


In [7]:
if option != 'denoiser_only':
    # 可视化一维梯度场对比（所有原子类型）
    atom_types = [0, 1, 2, 3, 4]  # C, H, O, N, F
    save_path = os.path.join(output_dir, f"field1d_sample_{sample_idx}")

    gradient_results = visualize_1d_gradient_field_comparison(
        gt_coords=gt_coords,
        gt_types=gt_types,
        converter=converter,
        field_func=field_func,
        sample_idx=0,  # 数据中只有1个样本，所以用索引0
        atom_types=atom_types,  # 传入列表，不需要循环
        x_range=None,
        y_coord=0.0,
        z_coord=0.0,
        save_path=save_path,  # save_path已经包含了正确的sample_idx (14441)
        display_sample_idx=sample_idx,  # 用于文件名和显示的原始样本索引
    )

    if gradient_results:
        print(f"Gradient field comparison (model: {model_dir}):")
        print(f"  Available atom types: {gradient_results['available_atom_types']}")
        
        # 打印每个原子类型的统计信息
        for atom_name, stats in gradient_results['all_results'].items():
            print(f"  {atom_name}: MSE={stats['mse']:.6f}, MAE={stats['mae']:.6f}")
            print(f"    Saved to: {stats['save_path']}")

elif option == 'denoiser_only':
    # 可视化denoiser生成的codes对应的梯度场在1维上的变化曲线
    print("\n=== 可视化1D梯度场（仅预测） ===")
    
    # 使用统一的场计算函数
    atom_types = [0, 1, 2, 3, 4]  # C, H, O, N, F
    save_path = os.path.join(output_dir, f"field1d_gen_sample_0")
    
    # 调用修改后的函数，不传入gt_coords和gt_types，只绘制预测的梯度场
    gradient_results = visualize_1d_gradient_field_comparison(
        gt_coords=None,  # 无ground truth
        gt_types=None,   # 无ground truth
        converter=None,  # 无ground truth时converter可以为None
        field_func=field_func,
        sample_idx=0,
        atom_types=atom_types,
        x_range=None,  # 使用默认范围(-5.0, 5.0)
        y_coord=0.0,
        z_coord=0.0,
        save_path=save_path,
        display_sample_idx=0,
    )
    
    if gradient_results:
        print(f"Gradient field visualization (generation mode):")
        print(f"  Available atom types: {gradient_results['available_atom_types']}")
        
        # 打印每个原子类型的统计信息
        for atom_name, stats in gradient_results['all_results'].items():
            print(f"  {atom_name}:")
            print(f"    Magnitude: Mean={stats.get('magnitude_mean', 'N/A'):.6f}, Std={stats.get('magnitude_std', 'N/A'):.6f}")
            print(f"    Saved to: {stats['save_path']}")


=== 可视化1D梯度场（仅预测） ===
使用默认 x 轴范围: (-11.0, 11.0)
Field 1D comparison (atom_type=C) saved to: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/field_1d_sample_0_atom_C.png
Field 1D comparison (atom_type=H) saved to: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/field_1d_sample_0_atom_H.png
Field 1D comparison (atom_type=O) saved to: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/field_1d_sample_0_atom_O.png
Field 1D comparison (atom_type=N) saved to: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/field_1d_sample_0_atom_N.png
Field 1D comparison (atom_type=F) saved to: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/field_1d_sample_0_atom_F.png
Gradient field v

In [8]:
if option == 'denoiser_only':
    # 对于 denoiser_only 模式，使用DDPM采样得到固定codes并可视化
    print("\n=== 执行 DDPM 分子生成 ===")

    grid_size = config.dset.grid_size
    code_dim = config.encoder.code_dim

    # 用于存储动画帧的列表和固定坐标轴限制
    frame_paths = []
    fixed_axis_limits_dict = {'limits': None}
    
    # 定义可视化回调函数
    def visualization_callback(iter_idx, all_points_dict, batch_idx):
        """在每次迭代时保存可视化帧"""
        # 合并所有原子类型的点
        all_points = []
        all_types = []
        for atom_type in range(5):  # C, H, O, N, F
            if atom_type in all_points_dict and len(all_points_dict[atom_type]) > 0:
                points = all_points_dict[atom_type]
                all_points.append(points)
                all_types.extend([atom_type] * len(points))
        
        if all_points:
            current_points = torch.cat(all_points, dim=0)
            current_types = torch.tensor(all_types, device=current_points.device)
        else:
            current_points = torch.empty((0, 3), device=codes.device)
            current_types = torch.empty((0,), device=codes.device, dtype=torch.long)
        
        # 如果是第一帧，确定固定坐标轴范围
        if iter_idx == 0 and len(current_points) > 0:
            points_np = current_points.detach().cpu().numpy()
            margin = 1.0
            fixed_axis_limits_dict['limits'] = {
                'x_min': points_np[:, 0].min() - margin,
                'x_max': points_np[:, 0].max() + margin,
                'y_min': points_np[:, 1].min() - margin,
                'y_max': points_np[:, 1].max() + margin,
                'z_min': points_np[:, 2].min() - margin,
                'z_max': points_np[:, 2].max() + margin
            }
        
        # 保存帧
        frame_path = os.path.join(output_dir, f"frame_gen_sample_0_{iter_idx:04d}.png")
        visualize_generation_step(
            current_points, iter_idx, frame_path, current_types, fixed_axis_limits_dict['limits']
        )
        frame_paths.append(frame_path)

    # 使用上一个单元已加载的 codes，重建分子，使用可视化
    print("Generating molecular field and reconstructing molecule (loaded codes)...")
    save_interval = 100
    recon_coords, recon_types = converter.gnf2mol(
        decoder,
        codes,
        fabric=fabric,
        save_interval = save_interval,
        visualization_callback = visualization_callback
    )
    
    print(f"Generated molecule: {recon_coords[0].shape[0]} atoms")
    print(f"Atom types: {recon_types[0].unique().tolist()}")

    # 创建 GIF 动画
    print("Creating generation process animation from saved frames...")
    import imageio
    gif_path = os.path.join(output_dir, f"funcmol_gen_sample_0.gif")
    with imageio.get_writer(gif_path, mode='I', duration=0.1, fps=15, loop=1) as writer:
        for frame_path in frame_paths:
            try:
                if not os.path.exists(frame_path):
                    print(f"Warning: Frame file {frame_path} does not exist, skipping...")
                    continue
                
                import time
                time.sleep(0.01)  # 短暂等待确保文件写入完成
                
                if os.path.getsize(frame_path) == 0:
                    print(f"Warning: Frame file {frame_path} is empty, skipping...")
                    continue
                
                frame = imageio.imread(frame_path)
                writer.append_data(frame)
            except Exception as e:
                print(f"Warning: Failed to read frame {frame_path}: {e}")
                continue
            finally:
                # 清理临时帧文件
                try:
                    if os.path.exists(frame_path):
                        os.remove(frame_path)
                except:
                    pass

    # 保存最终生成的分子
    final_path = os.path.join(output_dir, f"funcmol_gen_sample_0_final.png")
    # 过滤掉填充的原子（类型为-1的原子）
    valid_mask = recon_types[0] != -1
    if valid_mask.any():
        final_coords_valid = recon_coords[0][valid_mask]
        final_types_valid = recon_types[0][valid_mask]
        visualize_generated_molecule(
            final_coords_valid, final_types_valid, save_path=final_path
        )
    else:
        print("Warning: No valid atoms generated")

    print(f"\n=== DDPM Field 生成结果 ===")
    print(f"Generated atoms: {recon_coords[0].shape[0]}")
    print(f"Atom type distribution: {dict(zip(*torch.unique(recon_types[0], return_counts=True)))}")
    print(f"最终分子图: {final_path}")
    print(f"生成过程动画: {gif_path}")
    
else:
    # 根据option设置重建列表
    if option == 'gt_only':
        rec_list = ['gt_field']
    else:
        rec_list = ['predicted_field', 'gt_field']

    # 创建可视化器
    visualizer = GNFVisualizer(output_dir)

    # 为每种重建类型执行可视化
    for rec_type in rec_list:
        print(f"\n=== 执行 {rec_type} 重建 ===")
        
        # 根据重建类型设置场函数
        if rec_type == 'gt_field':
            # 使用统一的场计算函数
            field_func = create_field_function(
                mode='gt',
                converter=converter,
                gt_coords=gt_coords,
                gt_types=gt_types
            )
        else:  # predicted_field
            # 使用统一的场计算函数
            field_func = create_field_function(
                mode='predicted',
                decoder=decoder,
                codes=codes
            )
        
        # 执行重建可视化
        results = visualizer.create_reconstruction_animation(
            gt_coords=gt_coords,
            gt_types=gt_types,
            converter=converter,
            field_func=field_func,
            save_interval=100,
            animation_name=f"recon_sample_{sample_idx}_{rec_type}",
            sample_idx=0
        )

        print(f"\n=== {rec_type} 重建结果 ===")
        print(f"RMSD: {results['final_rmsd']:.4f}")
        print(f"Reconstruction Loss: {results['final_loss']:.4f}")
        print(f"KL Divergence (orig->recon): {results['final_kl_1to2']:.4f}")
        print(f"KL Divergence (recon->orig): {results['final_kl_2to1']:.4f}")
        print(f"GIF动画: {results['gif_path']}")
        print(f"对比图: {results['comparison_path']}")


=== 执行 DDPM(new) 分子生成 ===
Generating molecular field and reconstructing molecule (loaded codes)...
>>     Memory status at iteration 0: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 50: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 100: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 150: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 200: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 250: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 300: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 350: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 400: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 450: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 500: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at iteration 550: Allocated=0.02GB, Reserved=0.75GB
>>     Memory status at ite

/tmp/ipykernel_2950493/3564755995.py:83: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame = imageio.imread(frame_path)



=== DDPM Field 生成结果 ===
Generated atoms: 20
Atom type distribution: {tensor(0, device='cuda:0'): tensor(1, device='cuda:0'), tensor(1, device='cuda:0'): tensor(4, device='cuda:0'), tensor(2, device='cuda:0'): tensor(3, device='cuda:0'), tensor(3, device='cuda:0'): tensor(5, device='cuda:0'), tensor(4, device='cuda:0'): tensor(7, device='cuda:0')}
最终分子图: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/funcmol_gen_sample_0_final.png
生成过程动画: /datapool/data3/storage/pengxingang/pxg/hyc/funcmol-main-neuralfield/exps/funcmol/fm_qm9/20251111/model-epoch=399/funcmol_gen_sample_0.gif
